In [9]:

import pandas as pd
from pathlib import Path

In [10]:


# Define project and data paths 
project_root = Path(r"C:\Users\THINKPAD\Desktop\10_Academy\Week_10\ethiopia-fi-forecast")
data_folder = project_root / "data" / "raw"

# Excel files
unified_file = data_folder / "ethiopia_fi_unified_data.xlsx"
impact_file = unified_file  
reference_file = data_folder / "reference_codes.xlsx"
additional_file = data_folder / "Additional Data Points Guide.xlsx"

# Load Excel sheets 
# Load unified data sheet
data = pd.read_excel(unified_file, sheet_name="ethiopia_fi_unified_data", engine="openpyxl")

# Load impact links sheet
impact_links = pd.read_excel(impact_file, sheet_name="Impact_sheet", engine="openpyxl")

# Load reference codes
reference_codes = pd.read_excel(reference_file, engine="openpyxl")

# Load additional data guide
additional_data = pd.read_excel(additional_file, engine="openpyxl")

# Quick inspection 
print("Unified Data shape:", data.shape)
print("Impact Links shape:", impact_links.shape)
print("Reference Codes shape:", reference_codes.shape)
print("Additional Data Guide shape:", additional_data.shape)

# View first few rows
display(data.head())
display(impact_links.head())
display(reference_codes.head())
display(additional_data.head())


Unified Data shape: (43, 34)
Impact Links shape: (14, 35)
Reference Codes shape: (71, 4)
Additional Data Guide shape: (18, 8)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,IMP_0001,EVT_0001,impact_link,NaN,ACCESS,Telebirr effect on Account Ownership,NaN,NaN,15.0,NaN,...,increase,high,15.0,12,literature,Kenya,Example_Trainee,2025-01-20,NaN,Kenya M-Pesa showed +20pp over 5 years
1,IMP_0002,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on Telebirr Users,NaN,NaN,NaN,NaN,...,increase,high,NaN,3,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct subscriber acquisition
2,IMP_0003,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on P2P Transactions,NaN,NaN,25.0,NaN,...,increase,high,25.0,6,empirical,NaN,Example_Trainee,2025-01-20,NaN,New digital payment channel
3,IMP_0004,EVT_0002,impact_link,NaN,ACCESS,Safaricom effect on 4G Coverage,NaN,NaN,15.0,NaN,...,increase,medium,15.0,12,empirical,NaN,Example_Trainee,2025-01-20,NaN,Network investment from competition
4,IMP_0005,EVT_0002,impact_link,NaN,AFFORDABILITY,Safaricom effect on Data Affordability,NaN,NaN,-20.0,NaN,...,decrease,medium,-20.0,12,literature,Rwanda,Example_Trainee,2025-01-20,NaN,Competition typically reduces prices


,field,code,description,applies_to
0,record_type,observation,Actual measured value from a source,All
1,record_type,event,Policy launch market event or milestone,All
2,record_type,impact_link,Relationship between event and indicator (link...,All
3,record_type,target,Policy target or official goal,All
4,record_type,baseline,Starting point for comparison,All


,Unnamed: 0,Integrated Financial Access & Usage Index (IFAU index),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Alternative Baseline Surveys,NaN,NaN,NaN,NaN,NaN,NaN
2,B,Potential Direct Corelating Data Points,NaN,NaN,NaN,NaN,NaN,NaN
3,C,Potential Indirect (Enablers or Proxies) Corel...,NaN,NaN,NaN,NaN,NaN,NaN
4,D,Naunces and Market Contexts,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# List all columns in the unified dataset
print("Columns in Unified Data:")
print(data.columns.tolist())


Columns in Unified Data:
['record_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


Explore unified data

In [ ]:
# Count records by key fields
print("Counts by record_type:")
print(data['record_type'].value_counts(), "\n")

print("Counts by pillar:")
print(data['pillar'].value_counts(dropna=False), "\n")  # include NaNs

print("Counts by source_type:")
print(data['source_type'].value_counts(dropna=False), "\n")

print("Counts by confidence:")
print(data['confidence'].value_counts(dropna=False), "\n")

# Temporal range
print("Observation date range:")
print(data['observation_date'].min(), "to", data['observation_date'].max(), "\n")

# List unique indicators and coverage
print("Unique indicators and counts:")
print(data['indicator_code'].value_counts(), "\n")

# List unique events and dates using the available columns
events = data[data['record_type'] == 'event'][['category', 'observation_date']]
print(events.sort_values('observation_date'))



Counts by record_type:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64 

Counts by pillar:
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64 

Counts by source_type:
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64 

Counts by confidence:
confidence
high      40
medium     3
Name: count, dtype: int64 

Observation date range:
2014-12-31 00:00:00 to 2030-12-31 00:00:00 

Unique indicators and counts:
indicator_code
ACC_OWNERSHIP         7
ACC_FAYDA             4
USG_P2P_COUNT         2
ACC_MM_ACCOUNT        2
GEN_MM_SHARE          2
GEN_GAP_ACC           2
ACC_4G_COV            2
EVT_ETHIOPAY          1
EVT_MPESA_INTEROP     1
EVT_NFIS2             1
EVT_FX_REFORM         1
EVT_FAYDA             1
EVT_MPESA             1
EVT_SAFARICOM         1
EVT_TELEBIR

Explore Impact Links

In [14]:
# Count of impact links by parent_id and related_indicator
print("Impact links by parent_id:")
print(impact_links['parent_id'].value_counts(), "\n")

print("Impact links by related_indicator:")
print(impact_links['related_indicator'].value_counts(), "\n")


Impact links by parent_id:
parent_id
EVT_0001    3
EVT_0002    2
EVT_0003    2
EVT_0004    2
EVT_0007    2
EVT_0005    1
EVT_0008    1
EVT_0010    1
Name: count, dtype: int64 

Impact links by related_indicator:
related_indicator
USG_P2P_COUNT         3
AFF_DATA_INCOME       3
ACC_OWNERSHIP         2
USG_TELEBIRR_USERS    1
ACC_4G_COV            1
USG_MPESA_USERS       1
ACC_MM_ACCOUNT        1
GEN_GAP_ACC           1
USG_MPESA_ACTIVE      1
Name: count, dtype: int64 



Explore Reference Codes

In [15]:
print(reference_codes.head())
print(reference_codes.nunique())

         field         code  \
0  record_type  observation   
1  record_type        event   
2  record_type  impact_link   
3  record_type       target   
4  record_type     baseline   

                                         description applies_to  
0                Actual measured value from a source        All  
1            Policy launch market event or milestone        All  
2  Relationship between event and indicator (link...        All  
3                     Policy target or official goal        All  
4                      Starting point for comparison        All  
field          13
code           67
description    71
applies_to      5
dtype: int64
